<a href="https://www.kaggle.com/code/trakiet/duachuot?scriptVersionId=269190893" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/mooccubex-dataset/course-teacher.txt
/kaggle/input/mooccubex-dataset/course.json
/kaggle/input/mooccubex-dataset/paper.json
/kaggle/input/mooccubex-dataset/reply.json
/kaggle/input/mooccubex-dataset/concept-comment.txt
/kaggle/input/mooccubex-dataset/user-video.json
/kaggle/input/mooccubex-dataset/course-comment.txt
/kaggle/input/mooccubex-dataset/concept-problem.txt
/kaggle/input/mooccubex-dataset/user-reply.txt
/kaggle/input/mooccubex-dataset/school.json
/kaggle/input/mooccubex-dataset/translated_reply.json
/kaggle/input/mooccubex-dataset/video.json
/kaggle/input/mooccubex-dataset/concept-video.txt
/kaggle/input/mooccubex-dataset/other.json
/kaggle/input/mooccubex-dataset/course-school.txt
/kaggle/input/mooccubex-dataset/problem.json
/kaggle/input/mooccubex-dataset/user-comment.txt
/kaggle/input/mooccubex-dataset/user.json
/kaggle/input/mooccubex-dataset/exercise-problem.txt
/kaggle/input/mooccubex-dataset/concept-paper.txt
/kaggle/input/mooccubex-dataset/translated_com

In [2]:
BASE_DIR = '/kaggle/input/mooccubex-dataset/'
df_user_course_path = '/kaggle/input/user-course-v4/user_course_v4.csv'

In [3]:
import pyspark
from pyspark.sql import SparkSession
import warnings
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
import seaborn as sns
import matplotlib.pyplot as plt
from os import truncate
from pyspark.sql.functions import explode, col, sum, from_unixtime, hour, dayofweek, count, round, when, isnull, split, avg
warnings.filterwarnings("ignore")

spark = SparkSession.builder \
  .appName("reply_user_video_analysis") \
  .master("local[*]") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/19 15:44:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!ls {BASE_DIR}

comment-reply.txt    course-school.txt	      translated_reply.json
concept-comment.txt  course-teacher.txt       user-comment.txt
concept.json	     exercise-problem.txt     user.json
concept-other.txt    other.json		      user-problem.json
concept-paper.txt    paper.json		      user-reply.txt
concept-problem.txt  problem.json	      user-video.json
concept-reply.json   reply.json		      user-xiaomu.json
concept-video.txt    reply-reply.txt	      video_id-ccid.txt
course-comment.txt   school.json	      video.json
course-field.json    teacher.json
course.json	     translated_comment.json


# Video

In [5]:
df_video = spark.read.json(BASE_DIR + "/video.json")

In [6]:
df_video.show()

+--------------------+--------------------+----------------------+--------------------+------------------------------------+
|                ccid|                 end|                  name|               start|                                text|
+--------------------+--------------------+----------------------+--------------------+------------------------------------+
|0001603F826A3DDA9...|[4.255, 8.119, 10...|                 Video|[1.031, 7.095, 8....|[第二个就是短助记符在生成上面。有...|
|0003DB14A14A53279...|[12.581, 17.38, 1...|                 Video|[11.799, 12.581, ...|  [大家好, 今天我们讨论一维系统辐...|
|0004A5C6F07E369D9...|[20.83, 23.39, 26...|        家禽的消化系统|[19.2, 20.84, 23....|   [各位同学 大家好, 我是来自中国...|
|00059EBD1371A6959...|[23.14, 194.458, ...|    1.2 计算机硬件系统|[19.372, 190.522,...| [大家好 本视频介绍计算机硬件系统...|
|0005D1DC01B4EF8F9...|[25.08, 26.775, 2...|      什么是供应链网络|[24.33, 25.081, 2...|   [大家好, 今天我们学习的知识点,...|
|0007919ED7652A779...|[7.26, 9.97, 11.5...|    1.2 英语文字的起源|[6.29, 7.31, 10.0...|  [同学们好, 今天我们

# Video & ccid

In [7]:
df_video_id_ccid = spark.read.text(BASE_DIR + '/video_id-ccid.txt')
df_video_id_ccid.show(truncate=True)

+--------------------+
|               value|
+--------------------+
|V_234845\t0000363...|
|V_234876\t0000363...|
|V_234907\t0000363...|
|V_293392\t0000363...|
|V_293445\t0000363...|
|V_293494\t0000363...|
|V_627142\t0000363...|
|V_944346\t0000363...|
|V_944377\t0000363...|
|V_944408\t0000363...|
|V_1386166\t000036...|
|V_1479635\t000036...|
|V_1494506\t000036...|
|V_1494536\t000036...|
|V_1494566\t000036...|
|V_1494614\t000036...|
|V_1494625\t000036...|
|V_1494656\t000036...|
|V_1570352\t000036...|
|V_2070578\t000036...|
+--------------------+
only showing top 20 rows



In [8]:
# Parse mapping file
df_video_id_ccid_v2 = df_video_id_ccid.select(
    split(col("value"), "\t")[0].alias("video_id"),
    split(col("value"), "\t")[1].alias("ccid")
)
df_video_id_ccid_v2.show(5, truncate=False)

+--------+--------------------------------+
|video_id|ccid                            |
+--------+--------------------------------+
|V_234845|0000363DB5B6E0869C33DC5901307461|
|V_234876|0000363DB5B6E0869C33DC5901307461|
|V_234907|0000363DB5B6E0869C33DC5901307461|
|V_293392|0000363DB5B6E0869C33DC5901307461|
|V_293445|0000363DB5B6E0869C33DC5901307461|
+--------+--------------------------------+
only showing top 5 rows



# Join video & video_ccid

In [9]:
from pyspark.sql.functions import array_max
video_info = df_video_id_ccid_v2.join(df_video, on=['ccid'], how='left')
video_info = (
    video_info
    .select(
        "ccid",
        "video_id",
        array_max("end").alias("video_duration") 
    )
)
video_info.show(truncate=False)

+--------------------------------+---------+--------------+
|ccid                            |video_id |video_duration|
+--------------------------------+---------+--------------+
|4177BDE0E546941A9C33DC5901307461|V_232692 |NULL          |
|4177BDE0E546941A9C33DC5901307461|V_232809 |NULL          |
|4177BDE0E546941A9C33DC5901307461|V_232925 |NULL          |
|4177BDE0E546941A9C33DC5901307461|V_267275 |NULL          |
|4177BDE0E546941A9C33DC5901307461|V_267660 |NULL          |
|4177BDE0E546941A9C33DC5901307461|V_267811 |NULL          |
|4177BDE0E546941A9C33DC5901307461|V_673099 |NULL          |
|4177BDE0E546941A9C33DC5901307461|V_1294801|NULL          |
|4177BDE0E546941A9C33DC5901307461|V_2162579|NULL          |
|4177BDE0E546941A9C33DC5901307461|V_2162697|NULL          |
|4177BDE0E546941A9C33DC5901307461|V_2420789|NULL          |
|4177BDE0E546941A9C33DC5901307461|V_2421018|NULL          |
|4177BDE0E546941A9C33DC5901307461|V_6223770|NULL          |
|4177BDE0E546941A9C33DC5901307461|V_6631

# Course

In [10]:
df_course = spark.read.json(BASE_DIR + '/course.json')

In [11]:
from pyspark.sql import functions as F

# 🧩 2️⃣ Lọc tài nguyên là video (resource_id bắt đầu bằng "V_")
df_video_course_map = (
    df_course
    .withColumn(
        "video_resource",
        F.filter(F.col("resource"), lambda x: x["resource_id"].startswith("V_"))
    )
    .withColumn("video_struct", F.explode_outer(F.col("video_resource")))
    .select(
        F.col("id").alias("course_id"),
        F.col("video_struct.resource_id").alias("video_id")
    )
)

df_video_course_map.show()

+---------+--------+
|course_id|video_id|
+---------+--------+
| C_584313|   V_849|
| C_584313|   V_850|
| C_584313|   V_851|
| C_584313|   V_857|
| C_584313|   V_859|
| C_584313|   V_861|
| C_584313|   V_862|
| C_584313|   V_867|
| C_584313|   V_868|
| C_584313|   V_869|
| C_584313|   V_870|
| C_584313|   V_871|
| C_584313|   V_877|
| C_584313|   V_878|
| C_584313|   V_879|
| C_584313|   V_880|
| C_584313|   V_881|
| C_584313|   V_883|
| C_584313|   V_884|
| C_584313|   V_885|
+---------+--------+
only showing top 20 rows



 # Join Course & Video info

In [12]:
df_course_video_info = df_video_course_map.join(video_info, on=['video_id'], how='left')

In [13]:
df_course_video_info.show(truncate=False)

+---------+---------+--------------------------------+--------------+
|video_id |course_id|ccid                            |video_duration|
+---------+---------+--------------------------------+--------------+
|V_1224994|C_932180 |6F1E041C820CB2F59C33DC5901307461|559.091       |
|V_1224998|C_932180 |5B39CFD3815B4D2B9C33DC5901307461|511.922       |
|V_1225002|C_932180 |3213B53265D17AB69C33DC5901307461|767.554       |
|V_2961778|C_676975 |429EB7C97CA852AF9C33DC5901307461|NULL          |
|V_2962669|C_676975 |00B886C53A82289F9C33DC5901307461|NULL          |
|V_471505 |C_696863 |76168D418AC9437F9C33DC5901307461|NULL          |
|V_471511 |C_696863 |DE965C5A77AFC35D9C33DC5901307461|NULL          |
|V_471513 |C_696863 |DD9A75726B972E629C33DC5901307461|NULL          |
|V_471524 |C_696863 |37D7ACF856036DF39C33DC5901307461|NULL          |
|V_471536 |C_696863 |1EBB825CBB1FC9359C33DC5901307461|NULL          |
|V_471538 |C_696863 |1F6E41380D563BB69C33DC5901307461|NULL          |
|V_471539 |C_696863 

In [14]:
# 🧩 Gom các video theo từng khóa học
df_video_course_agg = (
    df_course_video_info
    .groupBy("course_id")
    .agg(
        F.collect_list("video_id").alias("course_video_list"),
        F.collect_list("ccid").alias("course_ccid_list"),
        F.round(F.sum("video_duration"), 3).alias("total_video_duration")
    )
)

df_video_course_agg.show(truncate=True)


+---------+--------------------+--------------------+--------------------+
|course_id|   course_video_list|    course_ccid_list|total_video_duration|
+---------+--------------------+--------------------+--------------------+
|C_1017355|[V_1617852, V_161...|[3192EED147F0E4C1...|                NULL|
|C_1123979|[V_1686167, V_168...|[64ACC01B39CD9DF9...|                NULL|
|C_1169292|[V_1723187, V_172...|[760FAA89D73B1D48...|                NULL|
|C_1320505|[V_2019584, V_201...|[80B562B88C536BE2...|                NULL|
|C_1410126|[V_2145127, V_214...|[AD4DE9970070160D...|           27092.155|
|C_1428968|[V_6381900, V_638...|[76E6A4AAF19F4C24...|                NULL|
|C_1429002|[V_2174282, V_278...|[C095115DFAB7A549...|                NULL|
|C_1429102|[V_2174511, V_217...|[FDCF1A127E5F2CF8...|                NULL|
|C_1438353|[V_2213400, V_221...|[41051E62B2642343...|                NULL|
|C_1469181|[V_2303570, V_230...|[B4C7D6314EC80366...|                NULL|
|C_1627979|[V_2841177, V_

In [15]:
df_user_course = spark.read.csv(df_user_course_path, header=True, inferSchema=True)
df_user_course.show()

+---------+---------+-----------+----------+----------+-------------+-------------------+----------+
|  user_id|course_id|certificate|start_date|  end_date|duration_days|        enroll_time|remain_day|
+---------+---------+-----------+----------+----------+-------------+-------------------+----------+
|  U_10000|C_2033958|        1.0|2020-09-04|2020-12-31|        118.0|2020-10-27 09:07:30|      65.0|
|U_1000129|C_1925903|        1.0|2020-08-12|2020-12-31|        141.0|2020-09-06 14:37:17|     116.0|
|U_1000129|C_1992970|        1.0|2020-08-27|2020-12-31|        126.0|2020-09-06 14:45:37|     116.0|
|U_1000129| C_680884|        1.0|2020-09-01|2020-12-31|        121.0|2020-10-16 10:41:09|      76.0|
|U_1000342| C_697791|        0.0|2020-01-20|2020-07-31|        193.0|2020-02-28 10:27:22|     154.0|
|U_1000663|C_2328510|        0.0|2020-11-18|2021-02-28|        102.0|2020-11-27 22:41:36|      93.0|
| U_100071| C_696601|        1.0|2019-12-25|2020-06-30|        188.0|2020-04-15 08:51:39|  

In [16]:
df_user_video = spark.read.json(BASE_DIR + '/user-video.json')
df_user_video.show(truncate=True)

+--------------------+-------+
|                 seq|user_id|
+--------------------+-------+
|[{[{190.0, 158843...|  U_112|
|[{[{506.0, 160406...|  U_150|
|[{[{4556.8, 15981...|  U_172|
|[{[{109.0, 159883...|  U_189|
|[{[{64.747, 15821...|  U_197|
|[{[{206.8, 159814...|  U_382|
|[{[{598.978027343...|  U_514|
|[{[{9.388, 160075...|  U_670|
|[{[{99.5, 1580808...|  U_783|
|[{[{454.848426, 1...|  U_796|
|[{[{231.0, 160258...|  U_817|
|[{[{567.2, 160078...|  U_981|
|[{[{595.0, 157961...| U_1071|
|[{[{9.7, 15989332...| U_1073|
|[{[{324.4, 158791...| U_1193|
|[{[{924.9, 158436...| U_1206|
|[{[{36.5, 1580867...| U_1323|
|[{[{391.5, 160091...| U_1361|
|[{[{130.4, 159765...| U_1368|
|[{[{1214.0, 16037...| U_1418|
+--------------------+-------+
only showing top 20 rows



In [17]:
# Explode the outer array in 'seq' to get each video sequence as a row
df_exploded = df_user_video.select("user_id", explode("seq").alias("video_sequence"))
# Select the video_id and explode the 'segment' array within each video sequence
df_exploded = df_exploded.select(
    "user_id",
    df_exploded["video_sequence"]["video_id"].alias("video_id"),
    explode(df_exploded["video_sequence"]["segment"]).alias("segment_details")
)
# Select the individual fields from the 'segment_details' struct
df_flattened = df_exploded.select(
    "user_id",
    "video_id",
    df_exploded["segment_details"]["start_point"].alias("start_point"),
    df_exploded["segment_details"]["end_point"].alias("end_point"),
    df_exploded["segment_details"]["speed"].alias("speed"),
    )
# Show the resulting DataFrame with separated columns
df_flattened.show(truncate=False)

+-------+---------+-----------+---------+-----+
|user_id|video_id |start_point|end_point|speed|
+-------+---------+-----------+---------+-----+
|U_112  |V_1395633|130.0      |190.0    |1.0  |
|U_112  |V_1395633|220.0      |250.0    |1.0  |
|U_112  |V_1395633|478.8      |508.8    |1.0  |
|U_112  |V_1395633|655.1      |692.55   |1.25 |
|U_112  |V_1395635|135.0      |170.0    |1.0  |
|U_112  |V_1395636|128.3      |180.8    |1.5  |
|U_112  |V_1395639|100.0      |106.25   |1.25 |
|U_112  |V_1395639|180.0      |186.25   |1.25 |
|U_112  |V_6210799|9.0        |22.25    |1.25 |
|U_112  |V_6210799|28.0       |271.5    |1.25 |
|U_112  |V_6210799|279.0      |331.5    |1.5  |
|U_112  |V_6210799|346.0      |356.0    |1.5  |
|U_112  |V_6210800|6.0        |13.5     |1.5  |
|U_150  |V_6224262|286.0      |506.0    |2.0  |
|U_172  |V_6432959|4552.1     |4556.8   |1.0  |
|U_189  |V_6334508|4.0        |109.0    |1.0  |
|U_189  |V_6334508|119.0      |254.0    |1.0  |
|U_189  |V_6334516|593.0      |598.0    

# Tính tổng thời lượng video người học đã xem trong mỗi khóa học

In [18]:
df_user_video_watch = (
    df_flattened
    .withColumn("watched_duration", F.col("end_point") - F.col("start_point"))
)

In [19]:
df_user_video_watch.show()

+-------+---------+-----------+---------+-----+-----------------+
|user_id| video_id|start_point|end_point|speed| watched_duration|
+-------+---------+-----------+---------+-----+-----------------+
|  U_112|V_1395633|      130.0|    190.0|  1.0|             60.0|
|  U_112|V_1395633|      220.0|    250.0|  1.0|             30.0|
|  U_112|V_1395633|      478.8|    508.8|  1.0|             30.0|
|  U_112|V_1395633|      655.1|   692.55| 1.25|37.44999999999993|
|  U_112|V_1395635|      135.0|    170.0|  1.0|             35.0|
|  U_112|V_1395636|      128.3|    180.8|  1.5|             52.5|
|  U_112|V_1395639|      100.0|   106.25| 1.25|             6.25|
|  U_112|V_1395639|      180.0|   186.25| 1.25|             6.25|
|  U_112|V_6210799|        9.0|    22.25| 1.25|            13.25|
|  U_112|V_6210799|       28.0|    271.5| 1.25|            243.5|
|  U_112|V_6210799|      279.0|    331.5|  1.5|             52.5|
|  U_112|V_6210799|      346.0|    356.0|  1.5|             10.0|
|  U_112|V

# phân tích hành vi xem video của từng người học

In [20]:
df_user_watched_summary = (
    df_user_video_watch
    .groupBy("user_id")
    .agg(
        # Tổng thời lượng xem
        F.round(F.sum("watched_duration"), 2).alias("total_watched_duration"),
        
        # Thời lượng trung bình mỗi lượt
        F.round(F.avg("watched_duration"), 2).alias("avg_watched_duration"),
        
        # Tốc độ phát trung bình có trọng số
        F.round(
            (F.sum(F.col("watched_duration") * F.col("speed")) / F.sum("watched_duration")), 
            3
        ).alias("avg_speed"),
        
        # Số video khác nhau
        F.countDistinct("video_id").alias("num_videos_watched"),
        
        # Tổng số lượt xem (số dòng)
        F.count("*").alias("num_sessions"),
        
        # Danh sách video đã xem
        F.collect_list("video_id").alias("video_list")
    )
)

In [21]:
df_user_watched_summary.show()

+----------+----------------------+--------------------+---------+------------------+------------+--------------------+
|   user_id|total_watched_duration|avg_watched_duration|avg_speed|num_videos_watched|num_sessions|          video_list|
+----------+----------------------+--------------------+---------+------------------+------------+--------------------+
| U_1000290|                2311.7|              144.48|      1.0|                 4|          16|[V_6280707, V_628...|
| U_1000454|                495.12|              247.56|      1.0|                 2|           2|[V_7396061, V_618...|
|U_10008027|                6278.0|              184.65|    1.224|                 6|          34|[V_6010431, V_605...|
|U_10011100|               32565.7|                5.19|      1.0|                17|        6275|[V_6012902, V_601...|
|U_10012258|               19257.1|               11.08|      2.0|                 7|        1738|[V_7386531, V_738...|
|U_10013620|                3199.7|     

# thời lượng mà người học đã hoàn thành trong mỗi khóa học họ tham gia

In [22]:
df_user_video_course = (
    df_user_video_watch
    .join(
        df_course_video_info.select("video_id", "course_id", "video_duration"),
        on="video_id",
        how="inner"   # chỉ giữ các video có khóa học
    )
)

In [23]:
df_user_course_watch = (
    df_user_video_course
    .groupBy("user_id", "course_id")
    .agg(
        F.round(F.sum("watched_duration"), 2).alias("total_watched_duration"),
        F.round(F.sum("video_duration"), 2).alias("total_video_duration")
    )
    .withColumn(
        "completion_rate",
        F.round(
            F.col("total_watched_duration") / F.col("total_video_duration") * 100,
            3
        )
    )
)


In [24]:
df_user_course_final = (
    df_user_course_watch
    .join(
        df_user_course.select(
            "user_id", "course_id", "certificate", 
            "start_date", "end_date", "duration_days", "remain_day"
        ),
        on=["user_id", "course_id"],
        how="left"
    )
)


In [25]:
df_user_course_final.select(
    "user_id",
    "course_id",
    "certificate",
    "total_watched_duration",
    "total_video_duration",
    "completion_rate",
    "duration_days",
    "remain_day"
).show(truncate=False)


25/10/19 15:49:05 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/10/19 15:49:05 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/10/19 15:49:05 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/10/19 15:49:05 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/10/19 15:49:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/10/19 15:49:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/10/19 15:49:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/10/19 15:49:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/10/19 15:49:07 WARN RowBasedKeyValueBatch: Calling spill() on

+----------+---------+-----------+----------------------+--------------------+---------------+-------------+----------+
|user_id   |course_id|certificate|total_watched_duration|total_video_duration|completion_rate|duration_days|remain_day|
+----------+---------+-----------+----------------------+--------------------+---------------+-------------+----------+
|U_12218036|C_1678971|NULL       |269.7                 |NULL                |NULL           |NULL         |NULL      |
|U_13010820|C_697025 |NULL       |734.9                 |2783.1              |26.406         |NULL         |NULL      |
|U_15381157|C_681460 |1.0        |1660.9                |6740.66             |24.64          |211.0        |159.0     |
|U_15865863|C_1628160|NULL       |5.0                   |NULL                |NULL           |NULL         |NULL      |
|U_17953797|C_948102 |1.0        |529.82                |NULL                |NULL           |109.0        |101.0     |
|U_2324194 |C_1774989|0.0        |380.55

In [26]:
from pyspark.sql import functions as F

# 1️⃣ Tạo bảng course → video list
df_course_video_list = (
    df_course_video_info
    .groupBy("course_id")
    .agg(
        F.collect_list("video_id").alias("course_video_list"),
        F.round(F.sum("video_duration"), 2).alias("total_video_duration")
    )
)

# 2️⃣ Tóm tắt người dùng → video list và các thông số
df_user_summary = (
    df_user_video_watch
    .groupBy("user_id")
    .agg(
        F.collect_list("video_id").alias("user_video_list"),
        F.round(F.avg("watched_duration"), 2).alias("avg_watched_duration"),
        F.round(
            (F.sum(F.col("watched_duration") * F.col("speed")) / F.sum("watched_duration")),
            3
        ).alias("avg_speed"),
        F.countDistinct("video_id").alias("num_videos_watched"),
        F.count("*").alias("num_sessions"),
        F.round(F.sum("watched_duration"), 2).alias("total_watched_duration")
    )
)

# 3️⃣ Bảng người học – khóa học
df_user_course_base = df_user_course.select(
    "user_id", "course_id", "certificate", "start_date", "end_date", "duration_days", "remain_day"
)

# 4️⃣ Hợp nhất dữ liệu và tính đặc trưng
df_final = (
    df_user_course_base
    .join(df_user_summary, on="user_id", how="left")
    .join(df_course_video_list, on="course_id", how="left")
    # Giao video giữa user và course
    .withColumn("common_videos", F.array_intersect("user_video_list", "course_video_list"))
    .withColumn("num_common_videos", F.size("common_videos"))
    # Thời lượng xem ước lượng trong khóa
    .withColumn(
        "estimated_watched_in_course",
        F.round(
            F.when(F.size("course_video_list") > 0,
                F.col("total_watched_duration") * (F.col("num_common_videos") / F.size("course_video_list"))
            ).otherwise(0.0),
            2
        )
    )
    # Tỷ lệ hoàn thành ước lượng
    .withColumn(
        "completion_rate",
        F.round(
            F.when(F.col("total_video_duration") > 0,
                F.col("estimated_watched_in_course") / F.col("total_video_duration")
            ).otherwise(0.0),
            3
        )
    )
    # ⚙️ Đặc trưng 2.3.2: user-video behavior features
    .withColumn(
        "unique_videos_ratio",
        F.round(
            F.when(F.size("course_video_list") > 0,
                F.col("num_videos_watched") / F.size("course_video_list")
            ).otherwise(0.0),
            3
        )
    )
    .withColumn(
        "rewatch_ratio",
        F.round(
            F.when(F.col("num_videos_watched") > 0,
                (F.col("num_sessions") / F.col("num_videos_watched")) - 1
            ).otherwise(0.0),
            3
        )
    )
    .withColumn(
        "avg_completion_rate_per_video",
        F.round(
            F.when(F.col("total_video_duration") > 0,
                F.col("total_watched_duration") / F.col("total_video_duration")
            ).otherwise(0.0),
            3
        )
    )
    .withColumn(
        "video_activity_ratio",
        F.round(
            F.when(F.col("duration_days") > 0,
                F.col("total_watched_duration") / F.col("duration_days")
            ).otherwise(0.0),
            3
        )
    )
    .select(
        "user_id", "course_id", "certificate",
        "start_date", "end_date", "duration_days", "remain_day",
        "user_video_list", "avg_watched_duration", "avg_speed",
        "num_videos_watched", "num_sessions", "total_watched_duration",
        "course_video_list", "total_video_duration",
        "num_common_videos", "estimated_watched_in_course", "completion_rate",
        "unique_videos_ratio", "rewatch_ratio",
        "avg_completion_rate_per_video", "video_activity_ratio"
    )
)

df_final.show(truncate=False)


+----------+---------+-----------+----------+----------+-------------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+---------+------------------+------------+----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [27]:
df_final.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- course_id: string (nullable = true)
 |-- certificate: double (nullable = true)
 |-- start_date: date (nullable = true)
 |-- end_date: date (nullable = true)
 |-- duration_days: double (nullable = true)
 |-- remain_day: double (nullable = true)
 |-- user_video_list: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- avg_watched_duration: double (nullable = true)
 |-- avg_speed: double (nullable = true)
 |-- num_videos_watched: long (nullable = true)
 |-- num_sessions: long (nullable = true)
 |-- total_watched_duration: double (nullable = true)
 |-- course_video_list: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- total_video_duration: double (nullable = true)
 |-- num_common_videos: integer (nullable = false)
 |-- estimated_watched_in_course: double (nullable = true)
 |-- completion_rate: double (nullable = true)
 |-- unique_videos_ratio: double (nullable = true)
 |-- rewatch_ratio

In [28]:
total = df_final.count()

null_ratio = df_final.select([
    (F.count(F.when(F.col(c).isNull(), 1)) / total * 100).alias(c) for c in df_final.columns
])
null_ratio.show(truncate=False)


+-------+---------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+--------------------+-----------------+------------------+-----------------+----------------------+------------------+--------------------+-----------------+---------------------------+-----------------+-------------------+-------------+-----------------------------+--------------------+
|user_id|course_id|certificate      |start_date       |end_date         |duration_days    |remain_day       |user_video_list  |avg_watched_duration|avg_speed        |num_videos_watched|num_sessions     |total_watched_duration|course_video_list |total_video_duration|num_common_videos|estimated_watched_in_course|completion_rate  |unique_videos_ratio|rewatch_ratio|avg_completion_rate_per_video|video_activity_ratio|
+-------+---------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+--------------------+-----------------+--

In [29]:
# Lưu ra file Parquet (được khuyến nghị, vì nhanh hơn và giữ nguyên kiểu dữ liệu)
df_final.write.mode("overwrite").parquet("/kaggle/working/df_final_output_parquet_v1")

print("✅ Đã lưu df_final vào thư mục /kaggle/working/")

✅ Đã lưu df_final vào thư mục /kaggle/working/


In [30]:
import os
os.listdir("/kaggle/working")

['df_final_output_parquet_v1', '__notebook__.ipynb']

In [31]:
import shutil

input_dir = "/kaggle/working/df_final_output_parquet_v1"
output_zip = "/kaggle/working/df_final_output_parquet_v1"

shutil.make_archive(output_zip, 'zip', input_dir)
print("✅ Đã nén file thành df_final_output_parquet_v1.zip")


✅ Đã nén file thành df_final_output_parquet_v1.zip
